In [1]:
import os
import pyspark
conf = pyspark.SparkConf()
# conf.set('spark.ui.proxyBase'
# , '/user/' + os.environ['JUPYTERHUB_USER'] + '/proxy/4041')
conf.set('spark.sql.repl.eagerEval.enabled', True)
conf.set('spark.driver.memory','4g')
sc = pyspark.SparkContext(conf=conf)
spark = pyspark.SQLContext.getOrCreate(sc)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/04 22:42:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/12/04 22:42:29 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/12/04 22:42:29 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


/Users/poojaakki/opt/anaconda3/lib/python3.9/site-packages/pyspark/sql/context.py:157: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [31]:
import pandas as pd
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import StringType

schema= StructType(
      [StructField('title',StringType(),True),
      StructField('text',StringType(),True),
      StructField('subject',StringType(),True),
      StructField('date',StringType(),True)])

df0 = pd.read_csv('/Users/poojaakki/Desktop/projects/Fake-News-Detection-Data-Analysis/datasets/Kaggle/fake.csv')
df1 = pd.read_csv('/Users/poojaakki/Desktop/projects/Fake-News-Detection-Data-Analysis/datasets/Kaggle/true.csv')
df_fake = spark.createDataFrame(df0,schema=schema)
df_true = spark.createDataFrame(df1,schema=schema)

In [32]:
# add a label to the data for fake news as 0 and true news as 1 and shuffle using rand
from pyspark.sql.functions import lit, rand

df_concat = df_true.withColumn('flag', lit(1)).union(df_fake.withColumn('flag', lit(0))).orderBy(rand())

In [33]:
df_concat =  df_concat.drop("date", "title")

df_concat

22/12/04 22:55:47 WARN TaskSetManager: Stage 13 contains a task of very large size (4983 KiB). The maximum recommended task size is 1000 KiB.
22/12/04 22:55:48 WARN TaskSetManager: Stage 14 contains a task of very large size (4983 KiB). The maximum recommended task size is 1000 KiB.


text,subject,flag
Tucker Carlson de...,left-news,0
BRUSSELS (Reuters...,worldnews,1
LONDON (Reuters) ...,worldnews,1
Barack Hussein Ob...,left-news,0
NEW YORK (Reuters...,worldnews,1
LONDON (Reuters) ...,worldnews,1
Remember that cut...,News,0
CAIRO (Reuters) -...,politicsNews,1
ROME (Reuters) - ...,worldnews,1
"AUSTIN, Texas/WAS...",politicsNews,1


In [34]:
# In order to avaoid the confusion convert everything to lowercase
from pyspark.sql.functions import col, split, lower, regexp_replace, length

df_concat = df_concat\
.withColumn("text", regexp_replace(lower(col("text")), r"[^0-9a-z]", " "))\
.where(length("text")>0)

#df_concat.show(200)

In [35]:
# remove stop words from it
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/poojaakki/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [36]:
import pyspark.sql.functions as F

df_concat = df_concat.withColumn('words',F.split(F.col('text'),' '))
# df_concat

In [37]:
from pyspark.ml.feature import StopWordsRemover 

stopwordsRemovalFeature = StopWordsRemover(inputCol="words", 
                   outputCol="words without stop").setStopWords(stop)

In [38]:
from pyspark.ml import Pipeline

stopWordRemovalPipeline = Pipeline(stages=[stopwordsRemovalFeature])
pipelineFitRemoveStopWords = stopWordRemovalPipeline.fit(df_concat)

In [39]:
df_concat = pipelineFitRemoveStopWords.transform(df_concat)

In [40]:
# convert array of strings to strings
from pyspark.sql.functions import concat_ws

df_concat = df_concat.withColumn("text1", concat_ws(" ", df_concat["words without stop"]))
# df_concat

In [41]:
df_concat = df_concat.drop("words","words without stop")

# df_concat

In [42]:
df_concat = df_concat.drop("text")

# df_concat

In [43]:
#remove space

from pyspark.sql.functions import udf
import pyspark.sql.functions as f
from pyspark.sql.types import ArrayType

def space_removal(f):
    clean_tokens = []
    for tok in f:
        if tok:
            clean_tokens.append(tok)
    return clean_tokens

udf_space_removal = udf(space_removal, ArrayType(StringType()))

df_concat = df_concat.withColumn('text1', udf_space_removal(f.col('text1')))

In [45]:
df_concat.show()

22/12/04 22:56:35 WARN TaskSetManager: Stage 19 contains a task of very large size (4983 KiB). The maximum recommended task size is 1000 KiB.


22/12/04 22:56:36 WARN TaskSetManager: Stage 20 contains a task of very large size (4983 KiB). The maximum recommended task size is 1000 KiB.


+------------+----+--------------------+
|     subject|flag|               text1|
+------------+----+--------------------+
|   left-news|   0|[t, u, c, k, e, r...|
|   worldnews|   1|[b, r, u, s, s, e...|
|   worldnews|   1|[l, o, n, d, o, n...|
|   left-news|   0|[b, a, r, a, c, k...|
|   worldnews|   1|[n, e, w,  , y, o...|
|   worldnews|   1|[l, o, n, d, o, n...|
|        News|   0|[r, e, m, e, m, b...|
|politicsNews|   1|[c, a, i, r, o,  ...|
|   worldnews|   1|[r, o, m, e,  ,  ...|
|politicsNews|   1|[a, u, s, t, i, n...|
|politicsNews|   1|[c, o, d, y,  ,  ...|
|   worldnews|   1|[i, s, l, a, m, a...|
|politicsNews|   1|[ , r, e, u, t, e...|
|politicsNews|   1|[s, a, n,  , f, r...|
|politicsNews|   1|[w, a, s, h, i, n...|
|politicsNews|   1|[w, a, s, h, i, n...|
|    politics|   0|[h, u, n, t, i, n...|
|    politics|   0|[b, u, m, p, y,  ...|
|   left-news|   0|[o, b, a, m, a,  ...|
|    politics|   0|[m, u, s, l, i, m...|
+------------+----+--------------------+
only showing top

In [24]:
df_concat = df_concat.withColumnRenamed("text1","text")

# df_concat

In [27]:
# most frequent words in fake news

df_fake = df_concat.filter(df_concat["flag"] == 0)

df_fake_text = df_fake.select("text")

# df_fake_text.show()

In [28]:
from pyspark.sql.functions import explode

df_fake_text = df_fake_text\
.withColumn("words", split(col("text"), " "))\
.where(length("text")>0)

df_freq_fake = df_fake_text\
.select(explode(col("words")).alias("word"))\
.groupBy("word").count()\

df_freq_fake.orderBy(col("count").desc()).show()

22/12/04 22:50:10 WARN TaskSetManager: Stage 6 contains a task of very large size (4983 KiB). The maximum recommended task size is 1000 KiB.


22/12/04 22:50:18 WARN TaskSetManager: Stage 7 contains a task of very large size (4983 KiB). The maximum recommended task size is 1000 KiB.


+---------+-------+
|     word|  count|
+---------+-------+
|         |1358755|
|    trump|  79352|
|     said|  33763|
|president|  27715|
|   people|  26570|
|      one|  24531|
|    would|  23562|
|  clinton|  19173|
|    obama|  18797|
|     like|  18098|
|   donald|  17681|
|     also|  15403|
|       us|  14822|
|     news|  14629|
|      new|  14394|
|  hillary|  14122|
|     even|  14011|
|     time|  13870|
|    state|  13465|
|    white|  13189|
+---------+-------+
only showing top 20 rows



In [30]:
# most frequent words in true news

df_true = df_concat.filter(df_concat["Flag"] == 1)

df_true_text = df_true.select("text")

# df_true_text.show()